In [1]:
"""
select one family and then put them into diff (valid) day.
if cost reduce, accept it; otherwise, accept by probability of exp(-cost_diff/temperature)
"""

import numpy as np
import pandas as pd

from util_io import init, finalize, dump_conf
from util_cost import cal_total, cal_diff_1, family_id_days_to_pref_cost, nd_ndp1_to_account_penality
from util_cost import n_people
from util_check import check_valid_all, check_valid_move
from util_events import move_family

In [2]:
from util_cost import family_id_days_to_pref_cost, nd_ndp1_to_account_penality

def cal_diff_1_accounting(occupancy, day0, day1, n):
    """ Difference in the accounting penalty for moving a family of size n from day0 to day1 
        given the current occupancy at day0.
    """
    # set of days that might affect their accounting penalties
    day_set = set([day0 - 1, day0, day1 - 1, day1])
    
    diff = 0.
    
    for day in day_set:
        if day == 100:
            diff -= nd_ndp1_to_account_penality[occupancy[day], occupancy[day]]
        elif day != 0:
            diff -= nd_ndp1_to_account_penality[occupancy[day], occupancy[day+1]]

    occupancy[day0] -= n
    occupancy[day1] += n
    
    for day in day_set:
        if day == 100:
            diff += nd_ndp1_to_account_penality[occupancy[day], occupancy[day]]
        elif day != 0:
            diff += nd_ndp1_to_account_penality[occupancy[day], occupancy[day+1]]

    occupancy[day0] += n
    occupancy[day1] -= n
    return diff
def cal_diff_1(family_id, occupancy, day0, day1):
    """ Difference in the toal cost for moving the family family_id from day0 to day1 
        given the current occupancy at day0.
        Note: family_id, day0, or day1 don't support vertorization
    """
    return (
        family_id_days_to_pref_cost[family_id, day1] - family_id_days_to_pref_cost[family_id, day0]
        + cal_diff_1_accounting(occupancy, day0, day1, n_people[family_id])
    )

In [3]:
# constants #
N_families = 5000
N_days = 100
# constants #

# params #
path_init_conf = '../output/m01-improved.csv'

temp_0 = 10
temp_1 = 11
temp_diff = 1

N_iters = 10000
N_dumps = 10000 # dump every iters
# params #

# init
assigned_day, family_on_day, occupancy = init(path_conf=path_init_conf)
etotal_low = cal_total(assigned_day, occupancy)
print('Init config cost:', etotal_low)

for temp in range(temp_0, temp_1, temp_diff):
    # init vars
    etotal = cal_total(assigned_day, occupancy)
    is_change = False
   
    print('At temp = %.3f. init cost: %.5f' % (temp, etotal))
    
    ediff_error_max = 1e-10

    for iters in range(N_iters):
        # pick family
        ifamily = np.random.randint(N_families)
        nfamily = n_people[ifamily]
    
        # pick day
        day0 = assigned_day[ifamily]
        day1 = np.random.randint(1, N_days+1)

        # check validity
        if day0 == day1:
            continue
        if not check_valid_move(occupancy, day0, day1, nfamily):
            continue

        # determine acceptance
        ediff = cal_diff_1(ifamily, occupancy, day0, day1)
        assigned_day_o, occupancy_o = assigned_day.copy(), occupancy.copy()
        move_family(ifamily, nfamily, day0, day1, assigned_day, family_on_day, occupancy)
        is_change = True
    
        # update etotal and check improvement
        etotal = etotal + ediff
        etotal_true = cal_total(assigned_day, occupancy)
        ediff_error = etotal - etotal_true
        print(iters, ediff_error)
        etotal = etotal_true
        if abs(ediff_error) > ediff_error_max:
            print(ifamily, nfamily, day0, day1)
            iters_max = iters
            assigned_day_o_max, occupancy_o_max = assigned_day_o.copy(), occupancy_o.copy()
            assigned_day_max, occupancy_max = assigned_day.copy(), occupancy.copy()
            ifamily_max, nfamily_max, day0_max, day1_max = ifamily, nfamily, day0, day1
            ediff_error_max = abs(ediff_error)

Read initial configs...
Read config completed.
Init config cost: 138848.5029529949
At temp = 10.000. init cost: 138848.50295
0 0.0
1 0.0
2 0.0
3 2.9103830456733704e-11
4 -2.9103830456733704e-11
5 0.0
6 0.0
8 0.0
9 -2.9103830456733704e-11
10 0.0
11 0.0
13 2.9103830456733704e-11
14 0.0
16 2.9103830456733704e-11
17 0.0
18 0.0
19 2.9103830456733704e-11
20 0.0
21 0.0
22 0.0
23 0.0
24 0.0
25 0.0
26 -2.9103830456733704e-11
27 0.0
28 0.0
29 0.0
30 0.0
31 -2.9103830456733704e-11
32 2.9103830456733704e-11
33 0.0
35 -2.9103830456733704e-11
36 2.9103830456733704e-11
37 0.0
38 -2.9103830456733704e-11
39 0.0
41 0.0
42 0.0
43 0.0
44 0.0
45 2.9103830456733704e-11
46 -2.9103830456733704e-11
48 2.9103830456733704e-11
49 0.0
50 0.0
51 0.0
52 0.0
53 0.0
54 -2.9103830456733704e-11
55 0.0
56 0.0
57 0.0
58 0.0
59 0.0
60 2.9103830456733704e-11
61 0.0
63 0.0
64 0.0
65 0.0
67 0.0
68 0.0
69 5.820766091346741e-11
70 -5.820766091346741e-11
71 0.0
72 5.820766091346741e-11
73 -5.820766091346741e-11
74 0.0
75 5.82076

907 0.0
908 0.0
909 0.0
910 0.0
911 -2.3283064365386963e-10
912 2.3283064365386963e-10
913 0.0
914 0.0
915 0.0
916 0.0
917 0.0
918 -2.3283064365386963e-10
919 0.0
920 0.0
921 0.0
922 2.3283064365386963e-10
923 0.0
924 0.0
925 -2.3283064365386963e-10
926 2.3283064365386963e-10
927 0.0
928 0.0
929 0.0
930 0.0
931 0.0
932 0.0
933 0.0
934 -2.3283064365386963e-10
936 0.0
937 0.0
938 0.0
939 0.0
940 0.0
941 0.0
942 0.0
943 0.0
944 0.0
945 0.0
946 0.0
947 0.0
948 2.3283064365386963e-10
949 0.0
950 0.0
951 0.0
952 0.0
953 0.0
954 -2.3283064365386963e-10
955 2.3283064365386963e-10
956 -2.3283064365386963e-10
957 0.0
958 2.3283064365386963e-10
959 0.0
960 0.0
961 0.0
962 0.0
963 0.0
964 0.0
965 2.3283064365386963e-10
966 0.0
967 0.0
968 0.0
969 0.0
970 0.0
971 2.3283064365386963e-10
972 0.0
973 -2.3283064365386963e-10
974 0.0
975 2.3283064365386963e-10
976 0.0
977 -2.3283064365386963e-10
978 2.3283064365386963e-10
979 0.0
980 0.0
981 0.0
982 0.0
983 0.0
984 0.0
985 -2.3283064365386963e-10
986 0.

1797 0.0
1798 0.0
1799 0.0
1800 0.0
1801 0.0
1802 0.0
1803 0.0
1804 -4.656612873077393e-10
1805 0.0
1806 0.0
1807 0.0
1808 -4.656612873077393e-10
1809 4.656612873077393e-10
1810 0.0
1811 0.0
1812 4.656612873077393e-10
1813 0.0
1814 0.0
1815 0.0
1816 -4.656612873077393e-10
1817 4.656612873077393e-10
1818 -4.656612873077393e-10
1819 0.0
1820 4.656612873077393e-10
1821 0.0
1822 0.0
1823 0.0
1824 0.0
1825 0.0
1826 -4.656612873077393e-10
1827 0.0
1828 4.656612873077393e-10
1829 -4.656612873077393e-10
1830 0.0
1831 0.0
1832 0.0
1833 0.0
1834 0.0
1835 0.0
1836 0.0
1837 4.656612873077393e-10
1838 0.0
1839 0.0
1840 0.0
1841 0.0
1842 0.0
1843 0.0
1844 -4.656612873077393e-10
1845 4.656612873077393e-10
1846 0.0
1847 0.0
1848 0.0
1849 0.0
1850 0.0
1851 4.656612873077393e-10
1852 -4.656612873077393e-10
1853 0.0
1854 0.0
1855 0.0
1856 0.0
1857 0.0
1858 0.0
1859 4.656612873077393e-10
1860 -4.656612873077393e-10
1861 0.0
1862 0.0
1863 0.0
1864 0.0
1865 0.0
1866 0.0
1867 0.0
1868 0.0
1869 0.0
1870 4.656

2773 0.0
2774 0.0
2775 0.0
2776 0.0
2777 0.0
2778 -9.313225746154785e-10
2779 0.0
2780 0.0
2781 0.0
2782 0.0
2783 9.313225746154785e-10
2784 0.0
2785 0.0
2786 0.0
2787 0.0
2788 0.0
2789 0.0
2790 9.313225746154785e-10
2791 -9.313225746154785e-10
2792 0.0
2793 0.0
2794 0.0
2795 0.0
2796 0.0
2797 -9.313225746154785e-10
2798 0.0
2799 0.0
2800 0.0
2801 0.0
2802 0.0
2803 0.0
2804 -9.313225746154785e-10
2805 0.0
2806 0.0
2807 0.0
2808 0.0
2809 9.313225746154785e-10
2810 0.0
2811 0.0
2812 0.0
2813 0.0
2814 0.0
2815 9.313225746154785e-10
2816 0.0
2817 0.0
2818 0.0
2819 0.0
2820 9.313225746154785e-10
2821 0.0
2822 -9.313225746154785e-10
2823 0.0
2824 0.0
2825 0.0
2826 0.0
2827 0.0
2828 0.0
2829 -9.313225746154785e-10
2830 0.0
2831 0.0
2832 9.313225746154785e-10
2833 0.0
2834 0.0
2835 0.0
2836 0.0
2837 0.0
2838 0.0
2839 -9.313225746154785e-10
2840 0.0
2841 0.0
2842 0.0
2843 0.0
2844 0.0
2845 9.313225746154785e-10
2846 0.0
2847 -9.313225746154785e-10
2848 9.313225746154785e-10
2849 0.0
2850 0.0
28

3710 0.0
3711 0.0
3712 0.0
3713 0.0
3714 0.0
3715 0.0
3716 0.0
3717 0.0
3718 -9.313225746154785e-10
3719 9.313225746154785e-10
3720 0.0
3721 0.0
3722 -9.313225746154785e-10
3723 0.0
3724 0.0
3725 0.0
3726 9.313225746154785e-10
3727 -9.313225746154785e-10
3728 0.0
3729 0.0
3730 9.313225746154785e-10
3731 0.0
3732 0.0
3733 0.0
3734 0.0
3735 9.313225746154785e-10
3736 -9.313225746154785e-10
3737 9.313225746154785e-10
3738 -9.313225746154785e-10
3739 9.313225746154785e-10
3740 0.0
3741 0.0
3742 0.0
3743 0.0
3744 9.313225746154785e-10
3745 -9.313225746154785e-10
3746 0.0
3747 0.0
3748 0.0
3749 0.0
3750 0.0
3751 -9.313225746154785e-10
3752 0.0
3753 9.313225746154785e-10
3754 0.0
3755 0.0
3756 0.0
3757 0.0
3758 0.0
3759 0.0
3760 0.0
3761 -9.313225746154785e-10
3762 0.0
3763 9.313225746154785e-10
3764 0.0
3765 0.0
3766 0.0
3767 0.0
3768 0.0
3769 0.0
3770 9.313225746154785e-10
3771 -9.313225746154785e-10
3772 0.0
3773 0.0
3774 0.0
3775 0.0
3776 0.0
3777 0.0
3778 0.0
3779 0.0
3780 0.0
3781 0.0
3

...

5591 0.0
5592 9.313225746154785e-10
5593 0.0
5594 -9.313225746154785e-10
5596 0.0
5597 -9.313225746154785e-10
5598 0.0
5599 9.313225746154785e-10
5600 0.0
5601 0.0
5602 -9.313225746154785e-10
5603 9.313225746154785e-10
5604 -9.313225746154785e-10
5605 0.0
5606 9.313225746154785e-10
5607 0.0
5608 -9.313225746154785e-10
5609 0.0
5610 0.0
5611 -9.313225746154785e-10
5612 0.0
5613 0.0
5614 0.0
5615 0.0
5616 0.0
5617 0.0
5618 9.313225746154785e-10
5619 -9.313225746154785e-10
5620 0.0
5621 0.0
5622 0.0
5623 -9.313225746154785e-10
5624 0.0
5625 9.313225746154785e-10
5626 0.0
5627 -9.313225746154785e-10
5628 9.313225746154785e-10
5629 0.0
5631 0.0
5632 0.0
5633 0.0
5635 0.0
5636 0.0
5637 0.0
5638 9.313225746154785e-10
5639 -9.313225746154785e-10
5640 9.313225746154785e-10
5641 0.0
5642 -9.313225746154785e-10
5643 9.313225746154785e-10
5645 -9.313225746154785e-10
5646 0.0
5647 0.0
5648 0.0
5649 9.313225746154785e-10
5650 -9.313225746154785e-10
5651 0.0
5653 0.0
5654 0.0
5655 0.0
5656 9.31322574

6432 0.0
6433 0.0
6434 -9.313225746154785e-10
6435 9.313225746154785e-10
6436 -9.313225746154785e-10
6437 0.0
6438 0.0
6439 9.313225746154785e-10
6440 0.0
6441 0.0
6442 0.0
6443 9.313225746154785e-10
6444 0.0
6445 0.0
6446 0.0
6447 -9.313225746154785e-10
6448 0.0
6449 0.0
6450 -9.313225746154785e-10
6451 0.0
6452 0.0
6453 -9.313225746154785e-10
6454 0.0
6455 0.0
6456 -9.313225746154785e-10
6457 0.0
6458 0.0
6459 0.0
6460 0.0
6461 0.0
6462 0.0
6463 0.0
6464 0.0
6465 0.0
6466 0.0
6467 0.0
6468 0.0
6470 0.0
6471 0.0
6472 0.0
6473 0.0
6474 0.0
6475 0.0
6476 0.0
6477 0.0
6478 0.0
6479 0.0
6480 0.0
6481 0.0
6482 0.0
6483 9.313225746154785e-10
6484 0.0
6485 0.0
6486 0.0
6487 0.0
6488 0.0
6489 -9.313225746154785e-10
6490 9.313225746154785e-10
6491 -9.313225746154785e-10
6492 0.0
6493 0.0
6494 0.0
6495 -9.313225746154785e-10
6496 0.0
6497 0.0
6498 0.0
6499 0.0
6500 0.0
6501 0.0
6502 0.0
6503 0.0
6504 0.0
6505 0.0
6506 0.0
6507 -9.313225746154785e-10
6508 0.0
6509 9.313225746154785e-10
6510 -9.3

7332 -9.313225746154785e-10
7333 0.0
7334 0.0
7335 0.0
7336 -9.313225746154785e-10
7337 0.0
7338 0.0
7339 9.313225746154785e-10
7340 0.0
7341 0.0
7342 -9.313225746154785e-10
7343 0.0
7344 9.313225746154785e-10
7345 0.0
7346 0.0
7347 0.0
7348 0.0
7349 0.0
7350 0.0
7351 0.0
7352 -9.313225746154785e-10
7353 0.0
7354 0.0
7355 0.0
7356 0.0
7357 0.0
7358 0.0
7359 0.0
7360 0.0
7361 -1.862645149230957e-09
7362 0.0
7363 0.0
7364 0.0
7365 -1.862645149230957e-09
7366 0.0
7367 1.862645149230957e-09
7368 0.0
7369 1.862645149230957e-09
7370 -1.862645149230957e-09
7371 0.0
7372 0.0
7373 1.862645149230957e-09
7374 0.0
7375 -1.862645149230957e-09
7376 0.0
7377 0.0
7378 0.0
7379 0.0
7380 1.862645149230957e-09
7381 0.0
7382 0.0
7383 0.0
7384 0.0
7385 0.0
7386 1.862645149230957e-09
7387 0.0
7388 -1.862645149230957e-09
7389 0.0
7390 1.862645149230957e-09
7391 -1.862645149230957e-09
7392 0.0
7393 1.862645149230957e-09
7394 -1.862645149230957e-09
7395 0.0
7396 0.0
7397 0.0
7398 1.862645149230957e-09
7399 0.0

8294 0.0
8295 0.0
8296 0.0
8297 0.0
8298 0.0
8299 0.0
8300 0.0
8301 0.0
8302 0.0
8303 0.0
8304 0.0
8305 0.0
8306 0.0
8307 0.0
8308 0.0
8309 0.0
8310 0.0
8311 0.0
8312 0.0
8313 0.0
8314 0.0
8315 0.0
8316 -1.862645149230957e-09
8317 1.862645149230957e-09
8318 0.0
8319 0.0
8321 0.0
8322 0.0
8323 -1.862645149230957e-09
8324 1.862645149230957e-09
8325 0.0
8326 0.0
8327 0.0
8328 0.0
8329 0.0
8330 0.0
8331 0.0
8332 -1.862645149230957e-09
8333 0.0
8334 0.0
8335 0.0
8336 0.0
8337 0.0
8338 1.862645149230957e-09
8339 -1.862645149230957e-09
8340 1.862645149230957e-09
8341 0.0
8342 0.0
8343 0.0
8344 0.0
8345 0.0
8346 -1.862645149230957e-09
8347 0.0
8348 0.0
8349 1.862645149230957e-09
8350 -1.862645149230957e-09
8351 0.0
8352 0.0
8353 1.862645149230957e-09
8354 0.0
8355 0.0
8356 0.0
8357 -1.862645149230957e-09
8358 0.0
8359 0.0
8360 0.0
8361 1.862645149230957e-09
8362 0.0
8363 0.0
8364 -1.862645149230957e-09
8365 1.862645149230957e-09
8366 0.0
8367 0.0
8368 1.862645149230957e-09
8369 0.0
8370 -1.862

9149 1.862645149230957e-09
9150 0.0
9151 0.0
9153 0.0
9154 0.0
9155 0.0
9156 1.862645149230957e-09
9157 0.0
9158 0.0
9159 -1.862645149230957e-09
9160 0.0
9161 0.0
9162 0.0
9163 0.0
9164 0.0
9165 0.0
9166 0.0
9167 0.0
9168 1.862645149230957e-09
9169 -1.862645149230957e-09
9170 0.0
9171 1.862645149230957e-09
9172 0.0
9173 0.0
9174 0.0
9175 0.0
9176 0.0
9177 -1.862645149230957e-09
9178 1.862645149230957e-09
9179 0.0
9180 0.0
9181 0.0
9182 -1.862645149230957e-09
9183 0.0
9184 1.862645149230957e-09
9185 0.0
9186 0.0
9187 0.0
9188 0.0
9189 -1.862645149230957e-09
9190 0.0
9191 0.0
9192 3.725290298461914e-09
9193 0.0
9194 -1.862645149230957e-09
9195 -1.862645149230957e-09
9196 0.0
9197 -1.862645149230957e-09
9198 0.0
9199 1.862645149230957e-09
9200 0.0
9201 0.0
9202 0.0
9203 0.0
9204 1.862645149230957e-09
9205 0.0
9206 0.0
9207 -1.862645149230957e-09
9208 0.0
9209 1.862645149230957e-09
9210 -1.862645149230957e-09
9211 0.0
9212 0.0
9214 0.0
9215 0.0
9216 0.0
9217 0.0
9218 0.0
9219 0.0
9220 0.0


In [5]:
ediff_error_max, iters_max, ifamily_max, nfamily_max, day0_max, day1_max

(2.9802322387695312e-08, 9917, 4646, 4, 78, 93)

In [6]:
print(occupancy_o_max[[day0_max, day1_max]])
print(occupancy_max[[day0_max, day1_max]])

[179 193]
[175 197]


In [7]:
cal_diff_1(ifamily_max, occupancy_max, day0_max, day1_max)

1964.302692656362

In [8]:
cal_diff_1_accounting(occupancy_o_max, day0_max, day1_max, nfamily_max)

-323.5647773061983

In [9]:
occupancy_max[[day0_max-1, day0_max, day0_max+1, day1_max-1, day1_max, day1_max+1]]

array([191, 175, 170, 234, 197, 247], dtype=int32)

In [10]:
cal_total(assigned_day_max, occupancy_max) - cal_total(assigned_day_o_max, occupancy_o_max)

1862.435222670436

In [11]:
family_id_days_to_pref_cost[ifamily_max, day1_max] - family_id_days_to_pref_cost[ifamily_max, day0_max]

2186

In [17]:
def cal_range_accounting(occupancy):
    """ Calculate contribution to accounting penality from part of occupancy """
    #assert len(occupancy) == N_days + 2
    #occupancy = occupancy[:-1]
    return np.sum(
        (occupancy[:-1] - 125) / 400 *
        occupancy[:-1] ** (
            0.5 + np.abs(np.diff(occupancy)) / 50
        )
    )

In [18]:
cal_range_accounting(occupancy_max[[day0_max-1, day0_max, day1_max, day1_max+1]])

77258.42222326259

In [19]:
cal_range_accounting(occupancy_o_max[[day0_max-1, day0_max, day1_max, day1_max+1]])

323485.88461760624

In [28]:
ture_acc_diff = cal_range_accounting(occupancy_max[[day0_max-1, day0_max, day1_max, day1_max+1]]) - cal_range_accounting(occupancy_o_max[[day0_max-1, day0_max, day1_max, day1_max+1]])
ture_acc_diff

-246227.46239434363

In [26]:
(
    cal_range_accounting(
        occupancy_o_max[[day0_max-1, day0_max, day0_max+1]] + 
        np.array([0, -nfamily_max, 0])
    ) +
    cal_range_accounting(
        occupancy_o_max[[day1_max-1, day1_max, day1_max+1]] +
        np.array([0, nfamily_max, 0])
    )
)

313386.7345532651

In [22]:
(
    cal_range_accounting(occupancy_o_max[[day0_max-1, day0_max, day0_max+1]]) +
    cal_range_accounting(occupancy_o_max[[day1_max-1, day1_max, day1_max+1]])
)

644387.2577033625

In [30]:
bug_acc_diff = (
    cal_range_accounting(
        occupancy_o_max[[day0_max-1, day0_max, day0_max+1]] + 
        np.array([0, -nfamily_max, 0])
    ) +
    cal_range_accounting(
        occupancy_o_max[[day1_max-1, day1_max, day1_max+1]] +
        np.array([0, nfamily_max, 0])
    ) -
    (
        cal_range_accounting(occupancy_o_max[[day0_max-1, day0_max, day0_max+1]]) +
        cal_range_accounting(occupancy_o_max[[day1_max-1, day1_max, day1_max+1]])
    )
)
bug_acc_diff

-331000.5231500974

In [32]:
bug_acc_diff - ture_acc_diff

-84773.06075575377

In [34]:
cal_range_accounting(occupancy_o_max[[day0_max, day1_max]] - np.array([nfamily_max, 0]))

149318.98062480416

In [ ]:
# finalize
print('Final score:', cal_total(assigned_day, occupancy))